In [1]:
from owlready2 import *
onto_path.append("/home/alice/")
onto = get_ontology("http://www.semanticweb.org/alice/ontologies/2023/3/prova#").load()

In [2]:
with onto:
    class Predicates(Thing):
        pass
    class Functions(Thing):
        pass
    class Actions(Thing):
        pass
    class Types(Thing):
        pass
    class Objects(Thing):
        pass
    class Parameters(Thing):
        pass

AllDisjoint([Predicates,Functions,Actions,Types,Objects,Parameters])

AllDisjoint([prova.Predicates, prova.Functions, prova.Actions, prova.Types, prova.Objects, prova.Parameters])

In [3]:
list(onto.classes())

[prova.Actions,
 prova.Predicates,
 prova.Functions,
 prova.Objects,
 prova.Types,
 prova.Parameters]

In [4]:
#with onto:
 #sync_reasoner()

In [5]:
with onto:

    class has_effect_predicates(ObjectProperty):
        domain    = [Actions]
        range     = [Predicates]

    class is_effect_predicates(ObjectProperty):
        domain    = [Predicates]
        range     = [Actions]
        inverse_property = has_effect_predicates

    class has_effect_function(ObjectProperty):
        domain    = [Actions]
        range     = [Functions]

    class is_effect_function(ObjectProperty):
        domain    = [Functions]
        range     = [Actions]
        inverse_property = has_effect_function

    class has_object(ObjectProperty):
        domain    = [Predicates]
        range     = [Objects]

    class is_object(ObjectProperty):
        domain    = [Objects]
        range     = [Predicates]
        inverse_property = has_object

    class has_type(ObjectProperty):
        domain    = [Objects]
        range     = [Types]

    class is_type(ObjectProperty):
        domain    = [Types]
        range     = [Objects]
        inverse_property = has_type

    class has_params_actions(ObjectProperty):
        domain    = [Parameters]
        range     = [Actions]

    class is_params_actions(ObjectProperty):
        domain    = [Actions]
        range     = [Parameters]
        inverse_property = has_params_actions

    class has_params_types(ObjectProperty):
        domain    = [Parameters]
        range     = [Types]

    class is_params_types(ObjectProperty):
        domain    = [Types]
        range     = [Parameters]
        inverse_property = has_params_types

In [6]:
with onto:
    class has_value(DataProperty, FunctionalProperty): #functional means that each one has a single value
        domain =[Functions]
        range = [float]
    class is_grounded(DataProperty, FunctionalProperty): #functional means that each one has a single value
        domain =[Predicates]
        range = [bool]
    class has_operator(DataProperty): #is not a functional, each predicate can have more than one operator
        #domain =[Predicates]
        range = [str]
    class has_order(DataProperty, FunctionalProperty): 
        domain =[Parameters]
        range = [int]

check inverse
a = Actions("go")
p = Predicates("at")
#a.has_effect_predicates.append(p)
p.is_effect_predicates=[a]
a.has_effect_predicates
p.is_effect_predicates
is_effect_predicates.domain

In [7]:
domain_path = "/home/alice/PROPER_LPG/"
if os.path.isfile(domain_path+'/prova_domain.pddl'):
    output_path=domain_path+'/prova_domain.pddl'
    with open(output_path, "r") as domain_file:
        raw_domain = domain_file.readlines()


In [8]:
last_action=""
actions=[]
for p in raw_domain:
    if (":durative-action" in p) or (":action" in p):
        if (":durative-action" in p):
            last_action=p.replace(":durative-action","").replace("( ","").replace("\n","")
        elif (":action" in p):
            last_action=p.replace(":action","").replace("( ","").replace("\n","")
        actions.append(last_action)
actions.pop(0)
actions_objects={}
for a in actions:
    actions_objects[a]=Actions(a)

In [9]:
#FIND TYPES AND ADD TO THE ONTOLOGY
types=[]
t=False
for p in raw_domain:
        if (")" in p):
            t=False
        if t==True:
            types.append(p.replace("\t","").replace("\n",""))
        if (":types" in p):
            t=True
types_objects={}
for t in types:
   types_objects[t]=Types(t)

In [10]:
#PREDICATES
predicates=[]
t=False
for p in raw_domain:
        if (":action" in p):
            t=False
        if t==True:
            if "?" in p:
                new=p[p.find("(")+1:p.find("?")].replace(" ","")
                
            else:
                new=p[p.find("(")+1:p.find(")")].replace(" ","")
                
            if new!="":   
                predicates.append(new)
        if (":predicates" in p):
            t=True
predicates_objects={}
for t in predicates:
    predicates_objects[t]=Predicates(t)


In [11]:
predicates_objects

{'production_room': prova.production_room,
 'assembly_room': prova.assembly_room,
 'presented_task': prova.presented_task,
 'at': prova.at,
 'human_present': prova.human_present,
 'block_to_deliver': prova.block_to_deliver,
 'empty_robot': prova.empty_robot,
 'finished': prova.finished,
 'extro': prova.extro,
 'intro': prova.intro,
 'computed_e': prova.computed_e,
 'consc': prova.consc,
 'unsc': prova.unsc,
 'computed_c': prova.computed_c,
 'agree': prova.agree,
 'disagree': prova.disagree,
 'computed_a': prova.computed_a}

In [12]:
functions=[]
t=False
for p in raw_domain:
        if (":predicates" in p):
            t=False
        if t==True:
            if "?" in p:
                new=p[p.find("(")+1:p.find("?")]
                
            else:
                new=p[p.find("(")+1:p.find(")")]
                
            if new!="":   
               functions.append(new)
        if (":functions" in p):
            t=True
function_objects={}
for t in functions:
    function_objects[t]=Functions(t)

In [13]:
#connect all the relations between actions predicates and functions
t=False
pr=False
last_action=""
new_params=[]
associated_types=[]
associated_actions=[]
associated_orders=[]
for p in raw_domain:
    if (":duration" in p) or (":precondition" in p):
        pr=False
    if (":durative-action" in p) or (":action" in p):
        t=False
        if (":durative-action" in p):
            last_action=p.replace(":durative-action","").replace("( ","").replace("\n","")
        elif (":action" in p):
            last_action=p.replace(":action","").replace("( ","").replace("\n","")
    if t==True:
        if ";" not in p:
            operation=p.replace("and","").replace("at end","").replace("("," ").replace(")"," ")
            pred=operation.replace("decrease","").replace("increase","").replace("assign","").replace("not","").split()
            #operation=operation.split()
           
            if pred==[]:
                continue
            else:
                   
               check=str(pred[0])
               for l in predicates:
                   if check==l:
                       actions_objects[last_action].has_effect_predicates.append(predicates_objects[check])
                       ops=last_action.replace(" ","")+" "+operation
                       predicates_objects[check].has_operator.append(ops)
                       break
                    
               for l in functions:
                   if check==l:
                       actions_objects[last_action].has_effect_function.append(function_objects[check])
                       ops=last_action.replace(" ","")+" "+operation
                       function_objects[check].has_operator.append(ops)
                       break
    if pr==True:
        params=p[p.index("(")+1:p.index(")")].replace("-","").split(" ")
        while 1:
                try:
                   params.remove("")
                except:
                   break
        c=0
        for i in params:
            if "?" in i:
                c+=1
                new_params.append(i)
                associated_actions.append(last_action)
                associated_orders.append(c)
            else:
                for j in range(c):
                    associated_types.append(i)
                c=0
    if ":parameters" in p:
        pr=True
    if ":effect" in p:
        t=True

In [14]:
parameters_objects={}
for t in new_params:
    parameters_objects[t]=Parameters(t)
    

In [15]:
for i in range(len(new_params)):
    parameters_objects[new_params[i]].has_params_actions.append(actions_objects[associated_actions[i]])
    if len(parameters_objects[new_params[i]].has_params_types)==0:
        parameters_objects[new_params[i]].has_params_types.append(types_objects[associated_types[i]])
    parameters_objects[new_params[i]].has_order=associated_orders[i]

In [16]:
parameters_objects["?l2"].has_params_types #different param types need different names

[prova.room]

In [17]:
for i in Functions.instances():
    print(i)

prova.interaction_level
prova.scrupulousness_level
prova.agreeableness_level
prova.dur
prova.no_blocks
prova.max_no_blocks
prova.extroversion_coefficient
prova.desired_interaction
prova.conscientious_coefficient
prova.desired_scrupulousness
prova.agreeableness_coefficient
prova.desired_agreeableness
prova.baseline


In [18]:
#initialize all the predicates to false and all the functions to 0
for p in predicates:
    predicates_objects[p].is_grounded=False
for f in functions:
    function_objects[f].has_value=0

In [19]:
if os.path.isfile(domain_path+'/prova_problem.pddl'):
    output_path=domain_path+'/prova_problem.pddl'
    with open(output_path, "r") as problem_file:
        raw_problem = problem_file.readlines()

In [20]:
objects=[]
associated_types=[]
t=False
for p in raw_problem:
        if (":init" in p):
            t=False
        if t==True:
            o_t=p.replace("\n","").replace(")","").replace("\t","").replace(" ","").split("-")
            if o_t[0] != "":
               print(o_t)
               objects.append(o_t[0])
               associated_types.append(o_t[1])
        if (":objects" in p):
            t=True


['r1', 'room']
['r2', 'room']


In [21]:
objects_objects={}
c=0
for t in objects:
    objects_objects[t]=Objects(t)
    objects_objects[t].has_type=[types_objects[associated_types[c]]]
    c+=1

In [22]:
#initialize the problem
t=False
for p in raw_problem:
        if (":goal" in p):
            t=False
        if t==True:
            prec=p.replace("("," ").replace(")"," ").replace("\n"," ").split(" ")
            while 1:
                try:
                    prec.remove("")
                except:
                    break
            if prec!=[]:
                if prec[0]=="=":
                    ff=prec[1]
                    function_objects[ff].has_value=float(prec[2])
                else:
                    pp=prec[0]
                    predicates_objects[pp].is_grounded=True
                    if len(prec)==2:
                         oj=prec[1]
                         predicates_objects[pp].has_object=[objects_objects[oj]]

                      
            
        if (":init" in p):
            t=True

In [23]:
onto.save()

In [24]:
#p=Predicates("finished")
#destroy_entity(p)

data properties
        int
        float
        bool
        str (string)
        owlready2.normstr (normalized string, a single-line string)
        owlready2.locstr (localized string, a string with a language associated)
        datetime.date
        datetime.time
        datetime.datetime

In [25]:
#plan the first time and get the list of action
planner_path = "/home/alice/PROPER_LPG/"
os.chdir (planner_path)
command =  './lpg++ -o prova_domain.pddl -f prova_problem.pddl -n 1 -force_neighbour_insertion'
#run the planner
fd_process = subprocess.Popen([command], stdout=subprocess.PIPE, shell=True)
(out, err) = fd_process.communicate()
fd_exit = fd_process.returncode


In [26]:
if os.path.isfile(planner_path+'/plan_prova_problem.pddl_1.SOL'):
        output_path=planner_path+'/plan_prova_problem.pddl_1.SOL'
        with open(output_path, "r") as plan_file:
            raw_plan = plan_file.readlines()
        plan=[]
        for p in raw_plan:
           if p[0] !=";":
            plan.append(p)
        ret=plan.remove("\n") 
        while ret==None:  
            try:
              ret=plan.remove("\n")
            except:
              ret="alice"    
        actions_to_execute=[]
        for i in plan:
            actions_to_execute.append(i[i.find("(")+1:i.find(")")])
        print(actions_to_execute)
        
else:
        print("Plan not found")

['CHECK_EXTROVERSION', 'CHECK_CONSCIENTOUS', 'CHECK_DISAGREEABLE', 'REACHING_PRODUCTION_ROOM R1 R2', 'COMPUTE_METRIC_CONSC', 'COMPUTE_METRIC_DISAGREE', 'COMPUTE_METRIC_EXTRO', 'PRESENT_PRODUCTION_ROOM R2', 'COMPUTE_METRIC_EXTRO', 'COMPUTE_METRIC_CONSC', 'APPROACH_HUMAN', 'CHECK_HUMAN_WORK', 'COMPUTE_METRIC_DISAGREE', 'ASK_PICK_THE_BLOCK R2', 'COMPUTE_METRIC_DISAGREE', 'COMPUTE_METRIC_EXTRO', 'COMPUTE_METRIC_CONSC', 'TRY_REPLACE_THE_HUMAN', 'REACHING_ASSEMBLY_ROOM R2 R1', 'COMPUTE_METRIC_EXTRO', 'COMPUTE_METRIC_DISAGREE', 'COMPUTE_METRIC_CONSC', 'PRESENT_ASSEMBLY_ROOM R1', 'COMPUTE_METRIC_EXTRO', 'APPROACH_HUMAN', 'COMPUTE_METRIC_CONSC', 'COMPUTE_METRIC_DISAGREE', 'REMIND_TO_PAY_ATTENTION', 'ASK_ASSEMBLY_BLOCK R1', 'COMPUTE_METRIC_EXTRO', 'COMPUTE_METRIC_CONSC', 'COMPUTE_METRIC_DISAGREE', 'REACHING_PRODUCTION_ROOM R1 R2', 'COMPUTE_METRIC_EXTRO', 'COMPUTE_METRIC_DISAGREE', 'COMPUTE_METRIC_CONSC', 'ASK_PICK_THE_BLOCK R2', 'COMPUTE_METRIC_DISAGREE', 'COMPUTE_METRIC_EXTRO', 'COMPUTE_METRIC_

In [37]:
def update_ontology(a):
    #check preconditions if neobjects_objects[params[parameters_objects[o[2]].has_order-1].lower()].has_typeeded
    ac=a.split(" ")[0]
    params=a.split(" ")[1:]
    preds=actions_objects[ac].has_effect_predicates
    #prendo tutti i predicati effetto di quell'azione
    for p in preds:
        ops=p.has_operator
        key = list(filter(lambda x: predicates_objects[x] == p, predicates_objects))[0]
        #per ogni predicato prendo tutte le operazioni di quel predicato
        for o in ops:
            o=o.replace("\n","").replace("\t","").split(" ")
            while 1:
                try:
                    o.remove("")
                except:
                    break
            #considero solo quelle che sono dell'azione che sto eseguendo
            if o[0]==ac:
                if o[1]=="not":
                    p.is_grounded=False
                    o.pop(1)
                else:
                    p.is_grounded=True
                
                if len(o)>2 and p.is_grounded==True:
                   print(p,[params[parameters_objects[o[2]].has_order-1].lower()])
                   #if parameters_objects[o[2]].has_params_types==objects_objects[params[parameters_objects[o[2]].has_order-1].lower()].has_type: #write the type of a èaram only when the predicate is set to true
                   p.has_object.append(objects_objects[params[parameters_objects[o[2]].has_order-1].lower()])


    funcs=actions_objects[ac].has_effect_function
    if funcs!=[]:
        for f in funcs:
            ops=f.has_operator
            key = list(filter(lambda x: function_objects[x] == f, function_objects))[0]
            #per ogni predicato prendo tutte le operazioni di quel predicato
            for o in ops:
                o=o.replace("\n","").replace("\t","").split(" ")
                while 1:
                    try:
                        o.remove("")
                    except:
                        break
                if o[0]==ac:
                    if o[1]=="assign":
                       f.has_value=int(o[3])
                    elif o[1]=="increase":
                        actual_value=f.has_value
                        if o[3]=="*":
                            v1=function_objects[o[4]].has_value
                            v2=function_objects[o[5]].has_value
                            f.has_value=actual_value + v1*v2
                        else:
                            f.has_value=actual_value + int(o[3])
                    elif o[1]=="decrease":
                        actual_value=f.has_value
                        if o[3]=="*":
                            v1=function_objects[o[4]].has_value
                            v2=function_objects[o[5]].has_value
                            f.has_value=actual_value - v1*v2
                        else:
                            f.has_value=actual_value - int(o[3])
                    else:
                        print("NO OPERATION FOUND")


In [38]:
for a in actions_to_execute:
    update_ontology(a)

prova.at ['r2']
prova.at ['r2']
prova.at ['r2']
prova.at ['r2']
prova.at ['r2']
prova.at ['r2']
prova.presented_task ['r2']
prova.presented_task ['r2']
prova.presented_task ['r2']
prova.presented_task ['r2']
prova.at ['r1']
prova.at ['r1']
prova.at ['r1']
prova.at ['r1']
prova.at ['r1']
prova.at ['r1']
prova.presented_task ['r1']
prova.presented_task ['r1']
prova.presented_task ['r1']
prova.presented_task ['r1']
prova.at ['r2']
prova.at ['r2']
prova.at ['r2']
prova.at ['r2']
prova.at ['r2']
prova.at ['r2']
prova.at ['r1']
prova.at ['r1']
prova.at ['r1']
prova.at ['r1']
prova.at ['r1']
prova.at ['r1']
prova.at ['r2']
prova.at ['r2']
prova.at ['r2']
prova.at ['r2']
prova.at ['r2']
prova.at ['r2']
prova.at ['r1']
prova.at ['r1']
prova.at ['r1']
prova.at ['r1']
prova.at ['r1']
prova.at ['r1']
prova.at ['r2']
prova.at ['r2']
prova.at ['r2']
prova.at ['r2']
prova.at ['r2']
prova.at ['r2']
prova.at ['r1']
prova.at ['r1']
prova.at ['r1']
prova.at ['r1']
prova.at ['r1']
prova.at ['r1']


In [39]:
predicates_objects["presented_task"].has_object

['r2',
 prova.r2,
 prova.r2,
 prova.r2,
 prova.r2,
 prova.r1,
 prova.r1,
 prova.r1,
 prova.r1]

In [ ]:
domain_path = "/home/alice/PROPER_LPG/"
if os.path.isfile(domain_path+'/prova_problem.pddl'):
    output_path=domain_path+'/prova_problem.pddl'
    with open(output_path, "r") as domain_file:
        raw_problem_copy= domain_file.readlines()
c=0
c_init_1=0
c_init_2=0
c_end_1=0
c_end_2=0
for p in raw_problem_copy:
    if "define" in p:
        c_init_1=c
    if "init" in p:
        c_init_2=c+1
    if "goal" in p:
        c_end_1=c-1
    c+=1   
c_end_2=c
domain_file.close()


In [ ]:
start_file=raw_problem_copy[c_init_1:c_init_2]

In [ ]:
end_file=raw_problem_copy[c_end_1:c_end_2]

In [ ]:
init_file=[]
for i in Predicates.instances():
    key = list(filter(lambda x: predicates_objects[x] == i, predicates_objects))[0]
    if i.is_grounded==True:
        new_line="      (" +key
        if i.has_object!=[]:
            key_obj = list(filter(lambda x: objects_objects[x] == i.has_object[0], objects_objects))[0]
            new_line=new_line+ " "+key_obj
        new_line=new_line+")\n"
        init_file.append(new_line)

In [ ]:
for i in Functions.instances():
    key = list(filter(lambda x: function_objects[x] == i, function_objects))[0]
    new_line="      (=("+key+")"+str(i.has_value)+")\n"
    init_file.append(new_line)

In [ ]:
new_pb=start_file+init_file+end_file
new_pb

In [ ]:
output_path=domain_path+'/prova_problem_copia.pddl'
with open(output_path, "w") as pb_file:
    for line in new_pb:
        pb_file.write(line)
        

In [ ]:
#replan
planner_path = "/home/alice/PROPER_LPG/"
os.chdir (planner_path)
command =  './lpg++ -o prova_domain.pddl -f prova_problem_copia.pddl -n 1 -force_neighbour_insertion'
#run the planner
fd_process = subprocess.Popen([command], stdout=subprocess.PIPE, shell=True)
(out, err) = fd_process.communicate()
fd_exit = fd_process.returncode